In [1]:
import tensorflow as tf

In [39]:
import numpy as np 
import math 
import copy 
import matplotlib
import matplotlib.pyplot as plt


In [5]:
input_size = 32 
latent_size = 16

In [6]:
model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape = (input_size, 1)),
            tf.keras.layers.Dense(input_size),
            tf.keras.layers.Dense(latent_size),
            tf.keras.layers.Dense(input_size),
            tf.keras.layers.Activation("sigmoid")
        ])

In [20]:

def hiff_fitness(array):
    """
    Calculate and return value related to h-iff 
    assignment to the binary string of array. 
    """

    def f(val):
        if val == 1 or val == 0:
            return 1
        else:
            return 0

    def t(left, right):
        if left == 1 and right == 1:
            return 1
        elif left == 0 and right == 0:
            return 0
        else:
            return None 

    def val_recursive(array, flor, sum):
        if flor > levels:
            return sum
        arr = []
        power = 2 ** flor
        for i in range(0,2**(levels - flor)-1,2):
            arr.append(t(array[i], array[i+1]))
            sum = sum + (f(array[i]) + f(array[i+1]))* power
        return val_recursive(arr, flor + 1, sum)

    size = len(array)
    if not (size/2).is_integer():
        raise ValueError("Array size must be power of 2.")
    levels = int(math.log2(size))
    sum = 0
    return val_recursive(array, 0,  sum)

def generate_training_sat(N, set_size):
    """
    Generate training set for H-IFF problem. 
    
    return: binary array of size N to train NN
    """
    input = np.ndarray(shape=(set_size, N))
    output = np.ndarray(shape=(set_size, N))

    if not (math.log2(N)).is_integer():
            raise ValueError("Array size must be power of 2.")
    for k in range(set_size):
        candidate_solution = np.random.randint(2, size = N)
        input[k]=candidate_solution
        solution_fitness = hiff_fitness(candidate_solution)
        for i in range(10 * N):
            index = np.random.randint(N)
            new_candidate_sol = copy.copy(candidate_solution)
            new_candidate_sol[index] = 1 - new_candidate_sol[index] # apply variation 
            new_fitness = hiff_fitness(new_candidate_sol) # check the change 
            if new_fitness >= solution_fitness : 
                candidate_solution = new_candidate_sol
                solution_fitness = new_fitness
        output[k]=candidate_solution

    return input, output

In [26]:
(trainX, trainY) = generate_training_sat(32, 1000)

(testX, testY) = generate_training_sat(32, 100)
opt = tf.keras.optimizers.Adam(lr=1e-3)

EPOCHS = 25
BS = 10

In [29]:


model.compile(loss="mse", optimizer=opt)

H = model.fit(
    trainX, trainY,
    validation_data = (testX, testY),
    epochs = EPOCHS,
    batch_size = BS
)


Epoch 1/25
100/100 [==============================] - 0s 2ms/step - loss: 0.2485 - val_loss: 0.2399
Epoch 2/25
100/100 [==============================] - 0s 1ms/step - loss: 0.2269 - val_loss: 0.2262
Epoch 3/25
100/100 [==============================] - 0s 1ms/step - loss: 0.2099 - val_loss: 0.2128
Epoch 4/25
100/100 [==============================] - 0s 1ms/step - loss: 0.1949 - val_loss: 0.2024
Epoch 5/25
100/100 [==============================] - 0s 1ms/step - loss: 0.1825 - val_loss: 0.1919
Epoch 6/25
100/100 [==============================] - 0s 1ms/step - loss: 0.1720 - val_loss: 0.1832
Epoch 7/25
100/100 [==============================] - 0s 1ms/step - loss: 0.1634 - val_loss: 0.1759
Epoch 8/25
100/100 [==============================] - 0s 1ms/step - loss: 0.1568 - val_loss: 0.1704
Epoch 9/25
100/100 [==============================] - 0s 1ms/step - loss: 0.1516 - val_loss: 0.1656
Epoch 10/25
100/100 [==============================] - 0s 1ms/step - loss: 0.1471 - val_loss: 0.1621

In [40]:
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

In [49]:
tmp = model.predict(testX)[0]
print(np.mean(tmp))
tmp = (tmp > 0.5).astype(np.int_)
print(tmp)

0.4940668
[0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 1 0 0 1 1 0 0 1 1 0 1 1 1 1 1 0 0]
